# Loading Libraries

In [ ]:
import os
import shutil
import random
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from torch.optim.lr_scheduler import ReduceLROnPlateau
from datetime import datetime
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score, fbeta_score

colors = ['#648E9C', '#9CB1BC', '#C5D4DE', '#E8F1F4']
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

# Functions

## Visualization

In [ ]:
# Function to plot samples
def plot_samples(images, N=5):
    ps = random.sample(range(0, images.shape[0]), N**2)
    f, axarr = plt.subplots(N, N, figsize=(10, 10))
    p = 0
    for i in range(N):
        for j in range(N):
            im = images[ps[p]].transpose(1, 2, 0)
            im = im / 2 + 0.5  # Unnormalize the image
            axarr[i, j].imshow(im)
            axarr[i, j].axis('off')
            p += 1
    plt.show()

## Train and Test

In [ ]:
def correct_predictions(predicted_batch, label_batch):
    pred = predicted_batch.argmax(dim=1, keepdim=True)
    return pred.eq(label_batch.view_as(pred)).sum().item()

def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    return total_params, trainable_params, non_trainable_params

def train_epoch(epoch, train_loader, network, optimizer, criterion, hparams, file_path):
    network.train()
    device = hparams['device']
    network.to(device)
    avg_loss = 0
    with open(file_path, 'a') as file:
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = network(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if batch_idx % hparams['log_interval'] == 0:
                log_str = (f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                           f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}\n')
                file.write(log_str)
                print(log_str, end='')  # print without adding an extra new line
    return avg_loss / len(train_loader)

def validation_epoch(validation_loader, network, hparams, file_path):
    network.eval()
    device = hparams['device']
    validation_loss = 0
    correct = 0
    with torch.no_grad(), open(file_path, 'a') as file:
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = network(data)
            validation_loss += F.cross_entropy(output, target, reduction='sum').item()
            correct += correct_predictions(output, target)
        validation_loss /= len(validation_loader.dataset)
        test_acc = 100. * correct / len(validation_loader.dataset)
        log_str = (f'\nTest set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} '
                   f'({test_acc:.0f}%)\n')
        file.write(log_str)
        print(log_str, end='')  # print without adding an extra new line
    return validation_loss, test_acc

## Plotting Results, Training and Testing Losses

In [ ]:
def plot_history(train_history, val_history, metric='accuracy'):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_history[metric])
    plt.plot(val_history[metric])
    plt.title(f'Training and Validation {metric}')
    plt.ylabel(metric)
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(train_history['loss'])
    plt.plot(val_history['loss'])
    plt.title('Training and Validation loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()

In [ ]:
def plot_and_save_results(tr_losses, te_losses, te_accs, model_name, dt_str):
    # Plotting Training and Testing Losses
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(tr_losses, label='Training Loss', color='blue')
    plt.plot(te_losses, label='Testing Loss', color='red')
    plt.title(f'Training and Testing Loss Over Epochs - Model: {model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plotting Testing Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(te_accs, label='Testing Accuracy', color='green')
    plt.title(f'Testing Accuracy Over Epochs - Model: {model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

    # Save the figure
    plt.savefig(f"{model_name}_{dt_str}_results.png")

    # Show the plot
    plt.show()

In [ ]:
def display_results(y_true, y_preds, class_labels):
    results = pd.DataFrame(precision_recall_fscore_support(y_true, y_preds)).T
    results.rename(columns={0: 'Precision',
                           1: 'Recall',
                           2: 'F-Score',
                           3: 'Support'}, inplace=True)
    
    conf_mat = pd.DataFrame(confusion_matrix(y_true, y_preds), 
                            columns=class_labels,
                            index=class_labels)

    f2 = fbeta_score(y_true, y_preds, beta=2, average='micro')
    accuracy = accuracy_score(y_true, y_preds)
    print(f"Accuracy: {accuracy}")
    print(f"Global F2 Score: {f2}")
    return results, conf_mat

def plot_confusion_matrix(conf_mat):
    plt.figure(figsize=(12, 10))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap=sns.color_palette("icefire", as_cmap=True), xticklabels=conf_mat.columns, yticklabels=conf_mat.index)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
def plot_predictions(images, y_true, y_preds, class_indices, num_samples=20):
    
    fig = plt.figure(figsize=(20, 10))
    for i, idx in enumerate(np.random.choice(len(images), size=num_samples, replace=False)):
        ax = fig.add_subplot(4, 5, i + 1, xticks=[], yticks=[])
        # Convert from PyTorch format (C, H, W) to image format (H, W, C)
        ax.imshow(images[idx].permute(1, 2, 0).cpu().numpy())
        pred_idx = y_preds[idx]
        true_idx = y_true[idx]
        
        # Set title with predicted and true labels
        ax.set_title(f"{class_indices[pred_idx]}\n(True: {class_indices[true_idx]})", 
                     color=("green" if pred_idx == true_idx else "red"))

    plt.tight_layout()
    plt.show()

## Setting-up Hyperparameters

In [ ]:
hparams = {
    'batch_size': 128,
    'num_epochs': 15,
    'learning_rate': 0.001,
    'weight_decay': 0.001,
    'log_interval': 10,
    'device': device,
    'num_classes': 10
}

## Brief Overview of the Data

In [ ]:
DATASET = "./data/EuroSAT_RGB"

LABELS = os.listdir(DATASET)
print(LABELS)

In [ ]:
# plot class distributions of whole dataset
counts = {}

for l in LABELS:
    counts[l] = len(os.listdir(os.path.join(DATASET, l)))

    
plt.figure(figsize=(12, 6))

plt.bar(range(len(counts)), list(counts.values()), align='center', color= colors[1])
plt.xticks(range(len(counts)), list(counts.keys()), fontsize=12, rotation=40)
plt.xlabel('Class Label', fontsize=13)
plt.ylabel('Class Size', fontsize=13)
plt.title('EUROSAT Class Distribution', fontsize=15);

In [ ]:
print(f'Dataset Size: {sum(counts.values())}')

The dataset is split into 10 classes of land cover. Each class varies in size, so I'll have to stratify later on when splitting the data into training, testing and validation sets.

In [ ]:
img_paths = [os.path.join(DATASET, l, l+'_1000.jpg') for l in LABELS]

img_paths = img_paths + [os.path.join(DATASET, l, l+'_2000.jpg') for l in LABELS]

def plot_sat_imgs(paths):
    plt.figure(figsize=(15, 8))
    for i in range(20):
        plt.subplot(4, 5, i+1, xticks=[], yticks=[])
        img = PIL.Image.open(paths[i], 'r')
        plt.imshow(np.asarray(img))
        plt.title(paths[i].split('/')[-2])

plot_sat_imgs(img_paths)


Upon reviewing the various class previews, we observe certain commonalities and distinct contrasts among them.

Classes depicting urban settings like `Highways`, `Residential` areas, and `Industrial` sites predominantly feature man-made structures and roadways.

Both `AnnualCrops` and `PermanentCrops` classes display agricultural areas, characterized by straight lines marking different crop fields.

In contrast, classes such as `HerbaceousVegetation`, `Pasture`, and `Forests` represent natural landscapes. `Rivers`, also a part of the natural landscape category, are somewhat more distinguishable from these other natural classes.

Analyzing the imagery content might give insights into potential class confusions. For instance, a `River` image could be misidentified as a `Highway`. Similarly, an image showing a highway junction with nearby buildings might be confused for an `Industrial` area. To overcome these challenges, it's crucial to develop a classifier that can adeptly discern these subtle differences.

Regarding Sentinel-2 satellite imagery, it's possible to access over 10 additional spectral bands. For instance, the `Near-Infrared Radiation (NIR)` band is available in this dataset and can be utilized to create indices that visualize the presence or absence of radiation in an image. However, this dataset lacks NIR wavelength bands, rendering this approach infeasible for our current analysis. Nonetheless, it's important to note that NIR data could offer an alternative method for tackling this classification task.

# Dataset

## Creating Train, Validation and Test Directories

In [ ]:
TRAIN_DIR = './data/train'
VALID_DIR = './data/valid'
TEST_DIR = './data/test'

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VALID_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# And open_project folder
os.makedirs(f'../open_project/', exist_ok=True)

# create class label subdirectories in train and test
for l in LABELS:
    os.makedirs(os.path.join(TRAIN_DIR, l), exist_ok=True)
    os.makedirs(os.path.join(VALID_DIR, l), exist_ok=True)
    os.makedirs(os.path.join(TEST_DIR, l), exist_ok=True)

## Splitting Data to Train and Test

In [ ]:
# Creating a dictionary with image paths and labels
data = {os.path.join(DATASET, l, img): l for l in LABELS for img in os.listdir(os.path.join(DATASET, l))}

X = pd.Series(list(data.keys()))
y = pd.get_dummies(pd.Series(data.values()))

# First split: Splitting into training and temporary set (combining test and validation)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=69)  # 30% for test + validation

for train_idx, temp_idx in split.split(X, y):
    train_paths = X[train_idx].tolist()
    temp_paths = X[temp_idx].tolist()

# Second split: Splitting the temporary set into test and validation
split_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=69)  # 50% of 30% -> 15% each for test and validation
y_temp = y.iloc[temp_idx]

for test_idx, val_idx in split_temp.split(pd.Series(temp_paths), y_temp):
    test_paths = pd.Series(temp_paths).iloc[test_idx].tolist()
    val_paths = pd.Series(temp_paths).iloc[val_idx].tolist()

# Define new paths without using regex
new_train_paths = [path.replace(DATASET, TRAIN_DIR) for path in train_paths]
new_test_paths = [path.replace(DATASET, TEST_DIR) for path in test_paths]
new_val_paths = [path.replace(DATASET, VALID_DIR) for path in val_paths]

train_path_map = zip(train_paths, new_train_paths)
test_path_map = zip(test_paths, new_test_paths)
val_path_map = zip(val_paths, new_val_paths)

## Uncomment Code Below when Run for First Time

In [ ]:
# Move the files
# print("Moving training files..")
# for old_path, new_path in tqdm(train_path_map):
#     os.makedirs(os.path.dirname(new_path), exist_ok=True)
#     shutil.copy(old_path, new_path)

# print("Moving testing files..")
# for old_path, new_path in tqdm(test_path_map):
#     os.makedirs(os.path.dirname(new_path), exist_ok=True)
#     shutil.copy(old_path, new_path)

# print("Moving validation files..")
# for old_path, new_path in tqdm(val_path_map):
#     os.makedirs(os.path.dirname(new_path), exist_ok=True)
#     shutil.copy(old_path, new_path)

## Loading and Preprocessing Train, Validation and Test Sets

In [ ]:
# Define transformations for training
mean = [0.485, 0.456, 0.406] # Adjust as needed
std = [0.229, 0.224, 0.225] # Adjust as needed

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(60),
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0), ratio=(0.75, 1.33)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)  
])

# Define transformations for testing (only normalization)
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)  
])

# Create the dataset and dataloader for training
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True)

# Optionally create the dataset and dataloader for validation
valid_dataset = datasets.ImageFolder(root=VALID_DIR, transform=test_transforms)
valid_loader = DataLoader(valid_dataset, batch_size=hparams['batch_size'], shuffle=False)

# Create the dataset and dataloader for testing
test_dataset = datasets.ImageFolder(root=TEST_DIR, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=hparams['batch_size'], shuffle=False)

In [ ]:
print(f'Train Dataset Size: {len(train_dataset)}')
print(f'Validation Dataset Size: {len(valid_dataset)}')
print(f'Test Dataset Size: {len(test_dataset)}')

In [ ]:
# Get Class indices dictionary
class_indices = train_dataset.class_to_idx
# Reverse the mapping to get indices to class names
idx_to_class = {v: k for k, v in class_indices.items()}
print(class_indices)
print(idx_to_class)

## Visualize the Preprocessed Data

In [ ]:
# Fetch the first batch of images from the trainloader
first_batch_images, _ = next(iter(train_loader))
first_image = first_batch_images[0].unsqueeze(0).to(hparams['device'])

In [ ]:
first_batch_images_np = first_batch_images.numpy()

# Plot samples from the first batch
plot_samples(first_batch_images_np)

# Model Instantiation

## EUROSatCNN

In [ ]:
class EUROSatCNN(nn.Module):
    def __init__(self, num_classes):
        super(EUROSatCNN, self).__init__()
        self.num_classes = num_classes

        # First set of convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.LeakyReLU()
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)

        # Second set of convolutional layers
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.LeakyReLU()
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool2d(2, 2) 
        self.dropout2 = nn.Dropout(0.25)

        # Third set of convolutional layers
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.relu5 = nn.LeakyReLU()
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.relu6 = nn.LeakyReLU()
        self.pool3 = nn.MaxPool2d(2, 2)  # output size: 8x8
        self.dropout3 = nn.Dropout(0.25)

        # Flatten layer
        self.flatten = nn.Flatten()

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # First FC layer
        self.relu7 = nn.LeakyReLU()
        self.dropout4 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(512, 128)  # New additional FC layer
        self.relu8 = nn.LeakyReLU()
        self.dropout5 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(128, self.num_classes)  # Final classification layer
    
    def forward(self, x):
        # First set of layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool1(x)
        x = self.dropout1(x)

        # Second set of layers
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        # Third set of layers
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu6(x)
        x = self.pool3(x)
        x = self.dropout3(x)

        # Flatten the output
        x = self.flatten(x)

        # Fully connected layers
        x = self.fc1(x)
        x = self.relu7(x)
        x = self.dropout4(x)

        x = self.fc2(x)
        x = self.relu8(x)
        x = self.dropout5(x)

        x = self.fc3(x)

        return x

## Function Instatiating Specific Model

In [ ]:
def compile_model(cnn_base, num_classes, file_path, fine_tune=None):
    # Choose the base model and modify the classifier
    if cnn_base in ['ResNet50', 'ResNet152V2']:
        if cnn_base == 'ResNet50':
            model = models.resnet50(pretrained=True)
        elif cnn_base == 'ResNet152V2':
            model = models.resnet152(pretrained=True)
        else:
            print("Wrong CNN base")
        
        # Modify the fully connected layer
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, num_classes)
        )
    
    elif cnn_base in ['VGG16', 'VGG19']:
        if cnn_base == 'VGG16':
            model = models.vgg16(pretrained=True)
        else:  # VGG19
            model = models.vgg19(pretrained=True)

        # Modify the classifier part of VGG
        num_ftrs = 512 * 7 * 7  # this is 25088 for 64x64 input images
        model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, num_classes)
        )
    elif cnn_base == "EUROSatCNN":
        model = EUROSatCNN(num_classes)

    # Set the fine-tuning
    if fine_tune is not None:
        for param in model.parameters():
            param.requires_grad = False
        for param in list(model.parameters())[fine_tune:]:
            param.requires_grad = True
    
    # Get the current datetime
    dt = datetime.now()

    # Format the datetime as a string in the specified format: year_month_day_hour_minute
    str_dt = dt.strftime("%Y_%m_%d_%H_%M")
    
    file_path = os.path.join(file_path, f'architecture_{str_dt}.txt')
    
    total_params, trainable_params, non_trainable_params = count_parameters(model)
    
    with open(file_path, 'a') as file:
        file.write('-------------------------------------------------------------------\n')
        file.write(f'{(model)}\n')
        file.write('-------------------------------------------------------------------\n')
        file.write(f'Total parameters in the model: {total_params}\n')
        file.write(f'Trainable parameters in the model: {trainable_params}\n')
        file.write(f'NON-Trainable parameters in the model: {non_trainable_params}\n')
        file.write('-------------------------------------------------------------------\n')

    return model, str_dt

# Training Process

In [ ]:
def training_process(network, nework_name, model_folder, str_dt):
    
    model_path = os.path.join(model_folder, f'best_model_{str_dt}.pth')
    
    train_log_path = os.path.join(model_folder, f'training_log_{str_dt}.txt')
    valid_log_path = os.path.join(model_folder, f'validation_log_{str_dt}_.txt')
    
    # Initialize variables for Early Stopping and Model Checkpoint
    best_val_accuracy = 0.0
    epochs_no_improve = 0
    patience = 10  # For early stopping
    

    # Define the optimizer with or without L2 regularization
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, network.parameters()), lr=hparams['learning_rate'], weight_decay=hparams['weight_decay'])

    # Define the criterion
    criterion = nn.CrossEntropyLoss()

    # Initialize ReduceLROnPlateau scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, min_lr=0.001)

    early_stopping_triggered = False


    # Training and validation loop
    tr_losses, te_losses, te_accs = [], [], []

    # Start the timer
    start_time = time.time()

    for epoch in range(1, hparams['num_epochs'] + 1):
        train_loss = train_epoch(epoch, train_loader, network, optimizer, criterion, hparams, train_log_path)
        tr_losses.append(train_loss)
        test_loss, test_acc = validation_epoch(valid_loader, network, hparams, valid_log_path)
        te_losses.append(test_loss)
        te_accs.append(te_accs)

        # Model Checkpoint
        if test_acc > best_val_accuracy:
            best_val_accuracy = test_acc
            torch.save(network.state_dict(), model_path)
            print(f"Checkpoint saved at {model_path}")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Early Stopping
        if epochs_no_improve == patience:
            print("Early stopping triggered. Exiting training loop.")
            early_stopping_triggered = True
            break

        # Learning Rate Scheduler Step
        scheduler.step(test_acc)

    if not early_stopping_triggered:
        print("Completed training for {} epochs".format(hparams['num_epochs']))
    
    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Convert elapsed time to minutes and seconds
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time % 60)

    print(f'Total Training Time: {elapsed_mins}m {elapsed_secs}s')
    
    plot_and_save_results(tr_losses, te_losses, te_accs, nework_name)
    
    return tr_losses, te_losses, te_accs, nework_name

# VGG16 No Fine Tuning

In [ ]:
# model_folder = '../open_project/vgg16_no_fine_tune_architecture'
# os.makedirs(model_folder, exist_ok=True)
# vgg16_no_fine_tune, str_dt = compile_model('VGG16', hparams['num_classes'], model_folder, fine_tune=None)

In [ ]:
# NO WAY THIS RUN ON MY LAPTOP
# vgg16_no_fine_tune_tr_losses, vgg16_no_fine_tune_te_losses, vgg16_no_fine_tune_te_accs, vgg16_no_fine_tune_nework_name = training_process(vgg16_no_fine_tune, "vgg16_no_fine_tune", str_dt)

# EUROSatCNN No Fine Tuning

In [ ]:
model_folder = '../open_project/EUROSatCNN_no_fine_tune'
os.makedirs(model_folder, exist_ok=True)

In [ ]:
EUROSatCNN_no_fine_tune, str_dt = compile_model('EUROSatCNN', hparams['num_classes'], model_folder, fine_tune=None)

In [ ]:
tr_losses, te_losses, te_accs, nework_name = training_process(EUROSatCNN_no_fine_tune, "EUROSatCNN_no_fine_tune", model_folder, str_dt)

# Model Evaluation

In [ ]:
# model = EUROSatCNN(hparams['num_classes'])
# model_path = "EUROSatCNN_no_fine_tune_model_best.pth"
# model.load_state_dict(torch.load(model_path, map_location=device))
# model.eval()

model = EUROSatCNN_no_fine_tune

In [ ]:
# Load a batch of images and their labels from the test_loader
images, labels = next(iter(test_loader))
images, labels = images.to(device), labels.to(device)

# Predict labels
model.eval()
model.to(device)
with torch.no_grad():
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

In [ ]:
# Map predicted and true labels back to class names
predicted_labels = [idx_to_class[idx] for idx in predicted.cpu().numpy()]
true_labels = [idx_to_class[idx] for idx in labels.cpu().numpy()]

# Visualize predictions
plot_predictions(images, true_labels, predicted_labels, class_indices, num_samples=20)

In [ ]:
# Initialize lists to store true and predicted labels
all_labels = []
all_predictions = []

# Predict labels for the entire test dataset
model.eval()
model.to(device)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Append predictions and labels to the lists
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

In [ ]:
# Now you can pass all_labels and all_predictions to the display_results function
prf, conf_mat = display_results(all_labels, all_predictions, class_indices)

In [ ]:
prf

In [ ]:
conf_mat

In [ ]:
# Use the function to plot the confusion matrix
plot_confusion_matrix(conf_mat)